In [1]:
# importing dependencies 
import pandas as pd
import numpy as np
from difflib import get_close_matches
import re
from config import api_key
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [2]:
# initalizing OpenCageGeocode
geocoder = OpenCageGeocode(api_key)

In [3]:
replace_list = [
    [
        "Between ",
        ""
    ],
    [
        "Subdistrict",
        "Sub District"
    ],
    [
        "Island Group",
        ""
    ],
    [
        "Provine",
        "Province"
    ],
    [
        " Atoll",
        ""
    ],
    [
        "miles from ",
        ""
    ],
    [
        "miles north of ",
        ""
    ],
    [
        "miles off ",
        ""
    ],
    [
        "miles southeast of ",
        ""
    ],
    [
        "miles west of ",
        ""
    ],
    [
        "miles SE of ",
        ""
    ],
    [
        "miles east of ",
        ""
    ],
    [
        "nm from ",
        ""
    ],
    [
        "nm southeast of ",
        ""
    ],
    [
        "nm st of ",
        ""
    ],
    [
        "to mileswest of the ",
        ""
    ],
    [
        "to miles ea off ",
        ""
    ],
    [
        " fresh water ",
        ""
    ],
    [
        " Pacific Ocean",
        ""
    ],
    [
        "miles offshore",
        ""
    ],
    [
        "Great Exuma Island",
        "Great Exuma"
    ],
    [
        "Guerrrero",
        "Guerrero"
    ],
    [
        "Guerro",
        "Guerrero"
    ],
    [
        "Great Exuma Island",
        "Great Exuma"
    ],
    [
        "Alinglaplap",
        "Alifu Alifu"
    ],
    [
        "Golfo di Genova in the ",
        ""
    ],
    [
        "Golfo de ",
        ""
    ],
    [
        "Inhambe",
        "Inhambane"
    ],
    [
        "Galica",
        "Galicia"
    ],
    [
        " between Port Edward and Port St Johns",
        ""
    ],
    [
        "Leyte Island",
        "Leyte"
    ],
    [
        "Liguaria",
        "Liguria"
    ],
    [
        "Ligurian Sea",
        "Liguria"
    ],
    [
        "miles southwest of O ahu ",
        ""
    ],
    [
        "the north coast",
        ""
    ],
    [
        "to Taboguilla Island Pacific Ocean",
        "Taboguilla Island"
    ],
    [
        "Corfu Island",
        "Corfu"
    ],
    [
        "Corfu ",
        "Corfu"
    ],
    [
        "Cyclades archipelago",
        "Cyclades"
    ],
    [
        "Enroute from Suez to Aden Yemen ",
        "Aden"
    ],
    [
        "nm west of the ",
        ""
    ],
    [
        "miles south of ",
        ""
    ],
    [
        "km off the coast of Iran km from mouth of ",
        ""
    ],
    [
        "San Blas coast",
        "San Blas"
    ],
    [
        "San Blas Islands",
        "San Blas"
    ],
    [
        "Vita Levu",
        "Viti Levu"
    ],
    [
        "Viti Levu Island",
        "Viti Levu"
    ],
    [
        "Viti Levu group",
        "Viti Levu"
    ],
    [
        "Vitu Levu",
        "Viti Levu"
    ],
    [
        " and U S A",
        ""
    ],
    [
        "East of the ",
        ""
    ],
    [
        "Off the coast of ",
        ""
    ],
    [
        "Off the western coast of peninsular ",
        ""
    ],
    [
        "Rombion",
        "Romblon"
    ],
    [
        "Tavenui",
        "Taveuni"
    ],
    [
        "Tuamotos",
        "Tuamotus"
    ],
    [
        "  province",
        ""
    ],
    [
        " arm of the North Sea",
        ""
    ],
    [
        " near Karitane north of Dunedin",
        ""
    ],
    [
        "Shat Al Arab River",
        "Shatt al Arab River"
    ],
    [
        "Shatt el Arab River",
        "Shatt al Arab River"
    ],
    [
        " les Bains",
        ""
    ],
    [
        "Saint Beno t",
        "Saint Benoit"
    ],
    [
        "Argyllshire",
        "Argyll"
    ],
    [
        "Makora Ulawa Province",
        "Makira Ulawa Province"
    ],
    [
        "Makira Uluwa Province",
        "Makira Ulawa Province"
    ],
    [
        "Somewhere between Philadelphia and ",
        ""
    ],
    [
        "Near the ",
        ""
    ],
    [
        "Near ",
        ""
    ],
    [
        "Off coast of ",
        ""
    ],
    [
        "Off ",
        ""
    ],
    [
        "the Coromandel Peninsula North Island",
        "Coromandel"
    ],
    [
        " in the Gulf of Leyte",
        ""
        
    ],
    
]

In [4]:
# def in_nested_list(my_list, item):
#     """
#     Determines if an item is in my_list, even if nested in a lower-level list.
#     """
#     return any(in_nested_list(sublist, item) for sublist in my_list if isinstance(sublist, list))

def clean_data(x):
    x = x.lstrip()
    x = re.sub("[^a-zA-Z]", ' ', x)
    x = re.sub(' +', ' ', x)
    x = x.lstrip()
    
    for sub_list in replace_list:
        x = x.replace(sub_list[0], sub_list[1])
    x = x.lstrip()
    x = x.rstrip()
    
    x = re.sub('^(to |To )', '', x)
    x = re.sub('^(Southern )', '', x)
    x = re.sub('^(East )', '', x)
    x = re.sub('^(Western )', '', x)
    x = re.sub('(Count)$', 'County', x)
    return x
    

In [14]:
# Cleaning national shark data
sharkAttacks = pd.read_csv('data/attacks.csv', encoding='cp1252').drop(columns=['Unnamed: 22', 'Unnamed: 23'])

sharkAttacks = sharkAttacks.dropna(subset=['Area'])
sharkAttacks['CleanArea'] = sharkAttacks.Area.apply(str)
sharkAttacks.CleanArea = sharkAttacks.CleanArea.apply(clean_data)

df1 = sharkAttacks.CleanArea + ", " + sharkAttacks.Country

df1 = df1.astype(str)

cleaned_areas_list = df1.unique().tolist()
cleaned_areas_list.sort()
# cleaned_areas_list


# len(thing)
# for t in thing:
#     print(f'{t} coordinates')

In [6]:
# Reading in txt file 
file_data = open("loc_data.txt", "r")
data = file_data.read()
# convert string from txt file to dictionary 
if data:
    location_data = eval(data)
else:
    # creating empty dict
    location_data = {}
    
# iterating through list cleaned_areas_list and checking if they are in location_data dict
# adding items that are not in dict to dict 
for item in cleaned_areas_list:
    query = f'{item}'
    if item not in location_data:
        location_data[item] = geocoder.geocode(query)
        
# writing dict to txt file
file = open("loc_data.txt","w")
file.write( str(location_data) )
file.close()

In [9]:
sharkAttacks

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,CleanArea,CombinedArea
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,California,"California, USA"
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,Georgia,"Georgia, USA"
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,Hawaii,"Hawaii, USA"
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,New South Wales,"New South Wales, AUSTRALIA"
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,Colima,"Colima, MEXICO"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6296,ND.0006,Before 1906,0.0,Unprovoked,AUSTRALIA,New South Wales,,Swimming,Arab boy,M,...,Said to involve a grey nurse shark that leapt ...,"L. Becke in New York Sun, 9/9/1906; L. Schultz...",ND-0006-ArabBoy-Prymount.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0006,ND.0006,7.0,New South Wales,"New South Wales, AUSTRALIA"
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,Australia,"Australia, AUSTRALIA"
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,Australia,"Australia, AUSTRALIA"
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,North Carolina,"North Carolina, USA"


In [27]:
file_data = open("loc_data.txt", "r")
data = file_data.read()

data = eval(data)
response_dict = {}
for key, value in data.items():
    if value:
        response_dict[key] = value[0].get("geometry")

# loc_df = pd.DataFrame(response_dict.items())
loc_df.reset_index()

loc_df.columns = ['CombinedArea', 'Geography']

sharkAttacks["CombinedArea"] = sharkAttacks.CleanArea + ", " + sharkAttacks.Country

# df.columns = ['a', 'b']

# sharkAttacks

# merged_df = pd.merge(sharkAttacks, loc_df, left_on='CleanArea', right_on='Area')
merged_df = sharkAttacks.merge(loc_df, left_on=['CombinedArea'], right_on=['CombinedArea'], how='left')


In [28]:
np.where(pd.isnull(merged_df))
unfound_location = merged_df[merged_df["Geography"].isna()][["Geography", "Country", "Location"]]
unfound_location

,Geography,Country,Location
103,NaN,"ST HELENA, British overseas territory",English Bay
142,NaN,"ST HELENA, British overseas territory",English Bay
854,NaN,TURKS & CAICOS,Mudjin Harbor
860,NaN,TURKS & CAICOS,French Cay
1732,NaN,TURKS & CAICOS,French Cay
2833,NaN,NaN,NaN
3245,NaN,NaN,Between St. Kitts & Nevis
3827,NaN,NaN,NaN
4034,NaN,NaN,NaN
4066,NaN,NaN,Geyser Bank


In [29]:
merged_df["Geography"][103] = "{'lat': -15.9655, 'lng': -5.7115}"
merged_df["Geography"][142] = "{'lat': -15.9655, 'lng': -5.7115}"
merged_df["Geography"][860] = "{'lat': 21.500, 'lng': -72.217}"
merged_df["Geography"][1732] = "{'lat': 21.500, 'lng': -72.217}"
merged_df["Geography"][854] = "{'lat': 21.788192, 'lng': -71.734542}"
merged_df["Geography"][3245] = "{'lat': 17.363747, 'lng': -62.754593}"
merged_df["Geography"][4131] = "{'lat': 40.946888, 'lng': 54.495243}"
merged_df["Geography"][4410] = "{'lat': 13, 'lng': 122}"
merged_df["Geography"][4744] = "{'lat': -38.73, 'lng': 77.522222}"
merged_df["Geography"][4281] = "{'lat': 15.748, 'lng': -81.75343}"
merged_df[merged_df["Geography"].isna()][["Geography", "Country", "Location"]]

/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/autumperconti/opt/anaconda3/envs/Project3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,Geography,Country,Location
2833,NaN,NaN,NaN
3827,NaN,NaN,NaN
4034,NaN,NaN,NaN
4066,NaN,NaN,Geyser Bank
4453,NaN,NaN,NaN
4459,NaN,NaN,NaN
5293,NaN,NaN,NaN
5403,NaN,NaN,NaN
5460,NaN,NaN,NaN
5726,NaN,NaN,Landing Place


In [34]:
reduced_merged = merged_df[['CleanArea', 'CombinedArea','Geography']]

reduced_merged = reduced_merged.drop_duplicates('CombinedArea').reset_index()
del reduced_merged['index']
reduced_merged

,CleanArea,CombinedArea,Geography
0,California,"California, USA","{'lat': 36.7014631, 'lng': -118.7559974}"
1,Georgia,"Georgia, USA","{'lat': 32.3293809, 'lng': -83.1137366}"
2,Hawaii,"Hawaii, USA","{'lat': 19.5872677, 'lng': -155.4268897}"
3,New South Wales,"New South Wales, AUSTRALIA","{'lat': -31.8759835, 'lng': 147.2869493}"
4,Colima,"Colima, MEXICO","{'lat': 19.166667, 'lng': -104.0}"
...,...,...,...
766,Bikini,"Bikini, MARSHALL ISLANDS","{'lat': 11.6243088, 'lng': 165.5505464}"
767,New Ireland New Britain,"New Ireland New Britain, PAPUA NEW GUINEA","{'lat': -5.7373, 'lng': 150.54406}"
768,Ba Ria Vung Tau Province,"Ba Ria Vung Tau Province, VIETNAM","{'lat': 10.5819798, 'lng': 107.2899841}"
769,Moala Island,"Moala Island, FIJI","{'lat': -18.6107187, 'lng': 179.8680733}"


In [40]:
# Writing to csv
reduced_merged.to_csv(r'geo_data.csv', index = False)